# Introduction

This is my notebook for the [2017 Advent of Code](http://adventofcode.com/2017/). I see this as an opportunity to write and share my thought processes, both for myself and anyone who comes across this (my students, hopefully). I also hope to note what skills are required for each problem, for my students and for my own memory. I remember students being concerned that these problems were outside their skill level, and I want to be able to directly point to the skills that I used to solve the problem and associate them with particular courses where possible.

Last year, I did not really attempt to compete, beyond attempting to complete all 25 days. However, based on my own casual timing with a stopwatch, I felt that I could have appeared in the rankings, so I am going to try to solve each problem this year at Midnight EST. This is typically past my bedtime, so we will see how this goes. I will be using Python throughout. I once considered Java to be my primary language, but I'm growing more comfortable with Python and I think Python is better suited to races against the clock (at least for writing code -- I know Java will actually solve the problem faster).

To begin with, I have included my solution template below. I built up this template over the course of the 2016 version, and after one day, I'm already finding things to change. Still, this will be my starting point:

In [ ]:
import itertools
import copy
import collections
import heapq
import math
import hashlib

f = open('input\\input__.txt', 'r')
lines = [line.strip() for line in f.readlines()]
f.close()

I used each of these modules multiple times. In fact, last year's Advent of Code introduced me, through colleagues and students, to the `itertools` module, which I find myself using more and more on challenges like these or Project Euler.

Inspired by Peter Norvig's template, I suspect I will add a few helper functions over the weekend. Most likely, an A* implementation is on the way, as I used that a few times last year. I suspect I will revise this template over time, but I will make a note in future days. Going deeper, this writeup in a Jupyter Notebook is also inspired by [Peter Norvig's notebook](https://github.com/norvig/pytudes/blob/master/ipynb/Advent%20of%20Code.ipynb), who I suspect will have better things to say about these problems than I will.

# [Day 1](http://adventofcode.com/2017/day/1): Inverse Captcha

This problem requires us to consider a single line of input -- a sequence of digits -- and find the sum of all digits that match the next digit in the sequence. That is, if the sequence includes `...55...`, then I should add 5 to my total to indicate the pair of 5s. If the sequence included `...555...`, I would add 5 twice, once for each adjacent pair. The problem also notes that we should consider the last digit of the sequence to be adjacent to the first digit of the sequence.

This is well within the capabilities of **Introduction to Programming** students. Specifically, my solution makes use of the *accumulator pattern*, *string indexing*, and *type conversion*. My solution is below:

In [1]:
def day1():
    f = open('input\\input1.txt', 'r')
    line = f.readline().strip()
    f.close()

    total = 0
    for idx in range(len(line)):
        if line[idx] == line[idx-1]:
            total += int(line[idx])

    return total

day1()

1089

This strategy takes advantage of one of my favorite, but simple, features of Python: negative indices. Instead of looking at the next digit in the sequence, my code actually checks the previous digit in the sequence. Same idea, but it means that when I start the loop at the first index of the sequence, I'm comparing to the digit at index -1 of the string. In other words, the first comparison happens between the first and last characters of the string, taking care of the tricky part of the problem right off the bat. From there, we compare each digit to the one that came immediately before it. When we have a match, we convert the digit to an integer and add it to our accumulator, `total`.

In **part two** of this puzzle, we no longer want to compare adjacent digits. Instead, we are told to compare each digit to the digit halfway through the sequence, looping back to the beginning if necessary. A useful provided sample input is `1212`. Each digit has a match. Looking at the first digit, we will have to compare to the third digit (since there are 4 digits total, we add 2 to our current comparison). These are a match. The same is true for the second digit. So our running total at this point is `1 + 2 = 3`.

Here is where I messed up, and it cost me a place in the rankings in part two: We have to compare **every** digit. So in our example from above, the third digit matches the fifth digit (which is just the first digit after we loop around). Same for the the fourth digit. So our final total is `1+2+1+2 = 6`. If we match a digit in the first half of the sequence, then we _must_ match the corresponding digit in the second half. But in my initial attempt, I assumed that we had already accounted for that. So my solution was incorrect, giving me half of the expected total. (The universe smacked me down again by having the power go out right after I got the wrong answer. So I blame not getting points in part two partially on my own misunderstanding, and partially on the power going out and distracting me. I ended up submitting my part two solution on my phone.)

My part two solution is below, which is based on my part one code:

In [2]:
def day1b():
    f = open('input\\input1.txt', 'r')
    line = f.readline().strip()
    f.close()

    length = len(line)
    half = length // 2

    total = 0
    for idx in range(len(line)):
        if line[idx] == line[(idx+half)%length]:
            total += int(line[idx])

    return total

day1b()

1156

The major change to this code comes in line 10, changing the right side of the comparison. Had I thought it through, I could have made use of negative indices, but modulo division came to mind first. Instead of comparing each index to the one before it, we compare the index to the one halfway through the sequence (represented by `idx+half`), modding by the length of the string to assure that we don't overshoot the end of the string.

# [Day 2](http://adventofcode.com/2017/day/2): Corruption Checksum

For **part one**, we are given a spreadsheet of integers. Our job is to find the difference between the minimum and maximum values in each row, and report the sum of the differences.

Again, this is solvable for any of my **Introduction to Programming** students, though my solution below uses some syntactic  techniques that I would not expect from CS1 students. Besides *file input*, the major techniques we need here are *type conversion*, *list iteration*, and the *accumulator pattern*.

My solution to part one is below:

In [3]:
def day2():
	f = open('input\\input2.txt', 'r')
	lines = [line.strip() for line in f.readlines()]
	f.close()

	total = 0
	for line in lines:
		data = [int(x) for x in line.split()]
		diff = max(data) - min(data)
		total += diff
		
	return total
	
	
day2()

47136

The list comprehension in Line 8 in this program does a lot of work. It divides each line at whitespace and converts each resulting string into an integer. So for each row, `data` is a list of integers in that row of the spreadsheet rather than a list of strings. Now that we have a list of integers, we can use `max()` and `min()` to painlessly calculate the difference between those two values.

For **part two**, we add a new wrinkle to the problem. Instead of locating the min and max in each row, we are told that there is one pair of values in each row such that one value evenly divides the other value of the pair. Once we find that pair, we should do the division and add that to our total.

My solution to this part is below, and it only adds some nested loops. It is not the cleanest solution, and I will address that below.

In [4]:
def day2b():
	f = open('input\\input2.txt', 'r')
	lines = [line.strip() for line in f.readlines()]
	f.close()

	total = 0
	for line in lines:
		data = [int(x) for x in line.split()]
		for i in range(len(data)):
			for j in range(i+1,len(data)):
				if data[i]%data[j] == 0:
					total += data[i]//data[j]
				elif data[j]%data[i] == 0:
					total += data[j]//data[i]
		
	return total

day2b()

250

The two inner `for` loops check each pair of values in a row to see if they divide evenly. If they do, we add the quotient to `total`.

Still, I had the `itertools` module in my template, but in my rush to solve the problem quickly, I ignored the module. Here is a slightly neater solution using the `combinations` function in `itertools`.

In [1]:
import itertools

def day2b_v2():
    f = open('input\\input2.txt', 'r')
    lines = [line.strip() for line in f.readlines()]
    f.close()

    total = 0
    for line in lines:
        data = [int(x) for x in line.split()]
        for i,j in itertools.combinations(data,2):
            if i%j == 0:
                total += i//j
            elif j%i == 0:
                total += j//i

    return total

day2b_v2()

250

# [Day 3](http://adventofcode.com/2017/day/3): Spiral Memory

Given a spiral strategy used to store data, identify the number of Manhattan-distance steps required to get from position 1 to the position represented by your input.

If you've done some problems in [Project Euler](https://projecteuler.net/), you've probably seen this spiral before. For my **part one** solution, I took advantage of a nice trait about the spiral. Here's the sample spiral we're given, filled out a few more steps:

`17  16  15  14  13
18   5   4   3  12
19   6   1   2  11
20   7   8   9  10
21  22  23  24  25`

Starting from the center (1) and working diagonally downward and to the right, we see the odd squares: 1, 9, 25. This pattern continues as the spiral grows. My strategy takes advantage of this, by finding the odd square that is closest to my input without going over. From there, we can identify it's location in the grid and continue filling out the spiral until we reach my input. Here is my solution, though as I note below, it's broken.

In [25]:
import math

def day3a(data):
    root = int(math.sqrt(data))
    loc = [root//2,-(root//2)]
    value = root**2 + 1
    height = -loc[1] + 1
    
    while value < data and loc[1]<height:
        loc[1] += 1
        value += 1
    while value < data and loc[0]>-height:
        loc[0] -= 1
        value += 1
    while value < data and loc[1]>-height:
        loc[1] -= 1
        value += 1
    while value < data and loc[0]<height:
        loc[0] += 1
        value += 1
    #print(loc)
    return (abs(loc[0])+abs(loc[1]))
    
day3a(265149)

438

Except my code doesn't exactly do what I said. It doesn't look for the nearest *odd* square -- it grabs the closest square period. For my input, that square just happens to be odd. As a result, there are other bugs in this solution. It doesn't pass the test inputs provided in the problem.

In [28]:
print(day3a(1) == 0)
print(day3a(12) == 3)
print(day3a(23) == 2)
print(day3a(1024) == 31)

True
False
False
False


In [32]:
import math

def day3a_v2(data):
    root = int(math.sqrt(data))
    if root % 2 == 0:
        root -= 1
    loc = [root//2,-(root//2)] # We're in the lower right corner of our grid.
    value = root**2
    max_xy = root//2 
    
    while value < data:
        # We need to expand the sprial now
        max_xy += 1
        
        # Let's take that next one step to the right to add root+1 to the spiral
        loc[0] += 1
        value += 1

        # From there we have to go up
        while value < data and loc[1]<max_xy:
            loc[1] += 1
            value += 1
        # Can't go up anymore, go left
        while value < data and loc[0]>-max_xy:
            loc[0] -= 1
            value += 1
        # Can't go left anymore, go down
        while value < data and loc[1]>-max_xy:
            loc[1] -= 1
            value += 1
        # Can't go down anymore, go right
        while value < data and loc[0]<max_xy:
            loc[0] += 1
            value += 1
        
        # Can't go right anymore. If we need to keep going, we need to loop back and expand.
    
    return abs(loc[0])+abs(loc[1])

print(day3a_v2(1))
print(day3a_v2(12))
print(day3a_v2(23))
print(day3a_v2(1024))
day3a_v2(265149)

0
3
2
31


438

This solution actually passes all of our test cases. Now we actually find the greatest odd square that is less than our data. Using the idea that 1 is at (0,0), we can keep track of our coordinates as we build the spiral from that odd square. To calculate the Manhattan distance, all we need to do is take the absolute value of our coordinates.

In terms of difficulty, this one kicked my butt a little bit. That said, nothing in my solution is outside of **Introduction to Programming.** I have some *nested loops*, and I am using some built-in functions, but there is nothing too advanced in this code.

The same is not true for **part two**, and I do not particularly love my solution. For one thing, I cannot use my part one solution to address this new wrinkle to the problem. For this part, we fill the grid by taking the sum of all adjacent spaces in the grid. My solution requires us to build the grid from scratch. Our goal is to identify the first number added to the grid that is larger than our input.

Let's look at this code in parts. First, here is my calcsum() function, which calculates the sum of the 8 adjacent neighbors, assuming that unfilled spaces have zeroes in them.

In [33]:
def calcsum(matrix, x, y):
    total = 0
    if x>=1 and y >= 1:
        total += matrix[x-1][y-1]
    if x>=1:
        total += matrix[x-1][y]
    if x>=1 and y<len(matrix)-1:
        total += matrix[x-1][y+1]
    if y >= 1:
        total += matrix[x][y-1]
    if y<len(matrix)-1:
        total += matrix[x][y+1]
    if x<len(matrix)-1 and y>=1:
        total += matrix[x+1][y-1]
    if x<len(matrix)-1:
        total += matrix[x+1][y]
    if x<len(matrix)-1 and y<len(matrix)-1:
        total += matrix[x+1][y+1]
    return total

My second function is the expand() function. It takes an NxN matrix and converts it to (N+2)x(N+2) by padding the entire outside with zeroes.

In [36]:
def expand(matrix):
    matlen = len(matrix)
    matrix.insert(0,[0]*(matlen+2))
    matrix.append([0]*(matlen+2))
    for row in range(1,matlen+1):
        matrix[row].insert(0,0)
        matrix[row].append(0)

Finally, the function below does all of the work.

In [43]:
def day3b(data):
    value = 1
    matrix = [[1]]
    expand(matrix)
    coords = [1,1]
    direction = 'R'
    
    while value<data:
        if direction == 'R':
            coords[0] += 1
            if coords[1] == len(matrix)-1 and coords[0] == len(matrix)-1:
                expand(matrix)
                coords[0]+=1
                coords[1]+=1
            elif coords[0] >= len(matrix)-1:
                direction = 'U'
        elif direction == 'L':
            coords[0] -= 1
            if coords[0] == 0:
                direction = 'D'
        elif direction == 'U':
            coords[1] -= 1
            if coords[1] == 0:
                direction = 'L'
        elif direction == 'D':
            coords[1] += 1
            if coords[1] == len(matrix)-1:
                direction = 'R'
        
        value = calcsum(matrix, coords[0], coords[1])
        
        matrix[coords[0]][coords[1]] = value
    return value

day3b(265149)

266330

Let's start before the while loop. Our most recent value is 1, which is at the center of a 1x1 matrix. However, we're immediately out of space, so we expand out to a 3x3 matrix. Within that matrix, our value (1) is now at `matrix[1][1]`, which is signified by `coords`. We also know that our next move is to the right.

The while loop continues so long as our last value is less than our provided input. Within that loop, we determine whether our move should be Right, Left, Up, or Down. In each case, the first step is to increase either the x- or y-coordinate. We then need to determine if we've hit an edge. If we're currently going Left, Up, or Down, then when we hit an edge, we just need to change direction. However, if we're going Right, there are two possibilities. Either we've reached the bottom right corner (in which case, the matrix is full), or we've just hit a right edge. If we hit the bottom right corner, we must expand and adjust our matrix coordinates. If we hit the right edge, we just need to change directions.

Once we've determined the direction of our next step, we can calculate the sum for this step and add it to the matrix.

Personally, I consider this solution ugly and inefficient. There is surely a better way to do this than to literally build the 2-dimensional matrix. I'm no longer certain that this code is within the realm of an **Introduction to Programming** student, mostly because of the shenanigans in the expand() function. I suspect that a **Data Structures** student could produce this code, though as I said, I think better solutions exist.

# [Day 4](http://adventofcode.com/2017/day/4): High-Entropy Passphrases

Given a list of passphrases, where each passphrase consisting of multiple words, determine the number of valid passphrases. A passphrase is not valid if it contains the same word multiple times.

For my **part one** solution, I divide each passphrase up into a list containing the words separately. I then convert that list to a set. If the lengths of the list and the set are the same (since sets cannot contain duplicates, but lists can), then the password is valid.

In [44]:
def day4a():
    f = open('input\\input4.txt', 'r')
    lines = [line.strip() for line in f.readlines()]
    f.close()
    
    total = 0
    for line in lines:
        passwords = line.split()
        pass_set = set(passwords)
        if len(passwords) == len(pass_set):
            total += 1
            
    return total
    
day4a()

383

To me, this problem sits on the border of **Introduction to Programming** and **Data Structures**. While we introduce sets in Introduction to Programming, I wonder if it wouldn't take until Data Structures for students to think of converting the list to a set in order to determine if there are no duplicates. That said, this problem is solvable for students in Introduction to Programming, but their solutions would likely be less efficient. I would expect a nested loop through the list of words to check for duplicates.

In **part two**, the rules are changed slightly. Now, a passphrase cannot contain words that are anagrams of each other. My solution is below:

In [46]:
def day4b():
    f = open('input\\input4.txt', 'r')
    lines = [line.strip() for line in f.readlines()]
    f.close()
    
    total = 0
    for line in lines:
        passwords = line.split()
        sorted_words = []
        for word in passwords:
            wordlist = list(word)
            sorted_words.append(''.join(sorted(wordlist)))
        pass_set = set(sorted_words)
        if len(sorted_words) == len(pass_set):
            total += 1
            
    return total
    
day4b()

265

Unlike Day 3, I get to build on my solution to the first part -- always a good sign in Advent of Code. Before we add the words to a set, I take each word and convert it to a list of characters. Then, I sort that list, and turn the list back into a string. Now I have `sorted_words`, a list of each word sorted by its letters. I build the set out of `sorted_words` rather than the basic `passwords`. Our validity check still compares list length against set length.

I have another version below that replaces the inner for loop with a nested list comprehension. Same steps, just compacted into one line. I'm sure I sacrifice some readability. But as I tried to build the solution, I was trying to come up with a list comprehension. Unfortunately, its structure wasn't obvious to me until I had written the loop solution above.

In [47]:
def day4b_comp():
    f = open('input\\input4.txt', 'r')
    lines = [line.strip() for line in f.readlines()]
    f.close()
    
    total = 0
    for line in lines:
        passwords = line.split()
        sorted_words = [''.join(sorted(wl)) for wl in [list(x) for x in passwords]]
        pass_set = set(sorted_words)
        if len(passwords) == len(pass_set):
            total += 1
            
    return total
    
day4b_comp()

265

In terms of difficulty, I think this part is exactly on par with the first part. My solution uses some **Data Structures** knowledge, but this problem is within reach of **Introduction to Programming** students.

# [Day 5](http://adventofcode.com/2017/day/5): A Maze of Twisty Trampolines, All Alike

Both parts of this problem require us to follow a series of numbers, which represent jumps. We need to keep count of how many jumps we make (including jumps of 0 steps) before we jump outside the bounds of the list. In **part one**, any time we jump from a particular number, we increment that number by 1. Solution below, followed by thoughts about it.

In [1]:
def day5a():
    f = open('input\\input5.txt', 'r')
    lines = [line.strip() for line in f.readlines()]
    f.close()
    
    jumps = [int(x) for x in lines]
    
    i = 0
    counter = 0
    exit = len(jumps)
    
    while i < exit:
        jumps[i] += 1
        i += (jumps[i] - 1)
        counter += 1
        
    return counter

day5a()

356945

This seemed like a particularly easy challenge, and one I would have no concern about giving to **Introduction to Programming** students. I will note that, while the concepts are easy and the required knowledge is pretty low (*arrays or lists* and *iteration* will suffice), there are opportunities to screw this up if steps are not handled in the exactly correct order. In a timed competition such as Advent of Code, that could hurt people if they make one of those mistakes.

Walking through my code, after reading the input, we convert the list of strings to a list of integers. I then set markers for my current index (`i`), a jump counter, and my threshold for when I've exceeded the bounds of the list (`exit`). I will note here that my code gets away with something, and I haven't researched whether this was true for all inputs: The problem definition states that you should terminate when you go outside the bounds of the list of numbers. My code checks when you fall off the far edge of the list, but I do not terminate if the index `i` ever becomes negative. This is an especially dangerous strategy considering how Python lets us use negative indices. I could correct this by changing my while loop condition to `while i >= 0 and i < exit:`.

Inside the while loop, I increment the jump value by 1, then increment `i` by the appropriate amount, correcting for the fact that I just incremented the jump value. Changing the order of these steps or missing the correction seem like places where people rushing to solve the problem could miss the solution. I'm lucky that I didn't get burned by ignoring one way to exceed the bounds of the list.

In **part two**, we change the rules for how to alter the jump value. Now, if the value was greater than or equal to 3, we decrement by 1. Otherwise, we continue incrementing. Solution below:

In [4]:
def day5b():
    f = open('input\\input5.txt', 'r')
    lines = [line.strip() for line in f.readlines()]
    f.close()
    
    jumps = [int(x) for x in lines]
    
    i = 0
    counter = 0
    exit = len(jumps)
    
    while i < exit:
        offset = jumps[i]
        if jumps[i] >= 3:
            jumps[i] -= 1
        else:
            jumps[i] += 1
        i += offset
        counter += 1
        
    return counter

day5b()

28372145

I'm in a good position again, because I get to build off my previous solution. The changes all come in the while loop. Besides adding the if-else block to handle the increment or decrement issue, I now save what the offset should be prior to making the change. This saves me from having to correct on line 18 the way I did in part one, which would be more difficult as I would have to keep track of whether I'm adjusting from an increment or a decrement. Also, I'm once again ignoring the edge case where `i` becomes less than zero, but I'm getting away with it.

# [Day 6](http://adventofcode.com/2017/day/6): Memory Reallocation

For this problem, our input is 16 integers, representing the number of blocks held by 16 memory banks. The reallocation procedure works by finding the memory bank with the largest load, reducing that load to zero, and distributing the blocks to the subsequent banks one at a time, looping back to the beginning of the list of blocks when necessary. We repeat this process for the bank that now has the largest load. For **part one**, the question we need to answer is how many reallocations have to occur before we reach a load state that we have already seen before.

Obviously, we need to keep track of which load states we have seen already. In my solution below, `banks` holds the current load in the 16 memory banks. I create a set, `observed`, that keeps track of which bank configurations we've already seen. We need to convert `banks` from a list to a tuple before we insert it into the set, since set contents need to be immutable. (Tuples are immutable, lists are mutable.)

Thus, the main loop continues so long as the current bank configuration is not in the observed set. When it's not, we add this (obviously new) configuration to the set, and search for the bank with the largest load. The for loop does this task, though in retrospect, it would have been much easier to replace the loop and the line before it with:
`index = banks.index(max(banks))`
I was rushing for the sake of the competition, and I thought of the loop first. Using the index() method didn't cross my mind until this morning when a colleague suggested it.

The remainder of the loop sets the overloaded bank to zero, and redistributes the load to the other banks.

In [2]:
def day6a():
    f = open('input\\input6.txt', 'r')
    lines = [line.strip() for line in f.readlines()]
    f.close()
    
    banks = [int(x) for x in lines[0].split()]
    
    observed = set()
    counter = 0
    while tuple(banks) not in observed:
        observed.add(tuple(banks))
        counter += 1
        
        index = -1
        for idx in range(len(banks)):
            if banks[idx] == max(banks):
                index = idx
                break
        
        distrib = banks[index]
        banks[index] = 0
        
        while distrib>0:
            index = (index+1)%len(banks)
            banks[index]+=1
            distrib-=1
    
    return counter
    
day6a()

5042

My solution uses concepts largely from **Introduction to Programming**. I could argue that truly understanding when/how to use tuples and sets might be a **Data Structures**-level expectation. If my students are reading this, then some of them are surely shocked at my use of `break`, considering how I admonish every student who thinks of using it. As I said above, I was speeding through for the sake of the competition. The better strategy would clearly have been using list's index() method, as I described.

For **part two**, we're asked a slightly different question. How many reallocations has it been since the first time we saw whichever configuration repeated. To address this, I added a dictionary, called `when` in the solution below. The keys to `when` are the bank configurations (in tuple format), while the values are the loop counter during the iteration where I saw it. Thus, I can change the return statement at the end to subtract the counter when I last saw the repeated configuration from the counter when I saw it the second time.

In [3]:
def day6b():
    f = open('input\\input6.txt', 'r')
    lines = [line.strip() for line in f.readlines()]
    f.close()
    
    banks = [int(x) for x in lines[0].split()]
    
    observed = set()
    when = dict()
    counter = 0
    while tuple(banks) not in observed:
        observed.add(tuple(banks))
        when[tuple(banks)] = counter
        counter += 1
        
        index = -1
        for idx in range(len(banks)):
            if banks[idx] == max(banks):
                index = idx
                break
        
        distrib = banks[index]
        banks[index] = 0
        
        while distrib>0:
            index = (index+1)%len(banks)
            banks[index]+=1
            distrib-=1
    
    return counter - when[tuple(banks)]
    
print(day6b())

1086


In fact, I could have replaced the `observed` set with the `when` dictionary entirely, rather than using both of them. The only other required change would be modifying the while loop condition to: `while tuple(banks) not in when:` The dictionary would do the entire job of the set. Still, for the sake of the competition, it was easier to keep both.

If I were to clean this code up, I would make two changes:
* In both part 1 and part 2, replace the for loop with `index = banks.index(max(banks))`
* In part 2, remove any reference to `observed`, since I can do the same job with the `when` dictionary

# [Day 7](http://adventofcode.com/2017/day/7): Recursive Circus

For this problem, our input is a list of "programs" which are supporting other programs in a tower. Each row of the list indicates one program in the tower, the weight of that program, and which other programs (if any) are directly supported by that program.

The tower is really a tree, so for the rest of this discussion, I will be using tree terminology. Each program is a node in the tree. By saying that a program supports several other programs, we are really saying that a parent node has several child nodes. Our input is essentially an adjacency list (typically a representation used for graphs, but perfectly valid for trees, as well). 

For **part one**, our goal is to identify the root of the tree. My solution (below) makes two passes through the input. In our first pass, we take the node identifier at the beginning of the line and add it to the `candidates` set -- that is, these nodes *could* be the root. In the second pass, we look only at the right side of the `-->` in the input so we can get the children of each node. If a node is a child (i.e. it has a parent), we remove it from the `candidates` set, since a root  has no parent. At the end, we *should* be left with exactly one identifier in the `candidates` set, though my code returns the entire set, rather than (hopefully) its sole element.

In [1]:
def day7a():
    f = open('input\\input7.txt', 'r')
    lines = [line.strip() for line in f.readlines()]
    f.close()

    candidates = set()
    for line in lines:
        parts = line.split()
        name = parts[0]
        candidates.add(name)
    for line in lines:
        parts = line.split()
        if len(parts) >= 3:
            for p in parts[3:]:
                p = p.rstrip(',')
                candidates.remove(p)
    return candidates
    
day7a()

{'ykpsek'}

For **part two**, we start to deal with those weights. We are told -- translated into tree terminology -- that, for any given node in the tree, the combined weights of each of that node's subtrees must be equal. We are further told that there is **exactly one node** whose weight is incorrect, violating that standard. Our output should be the **correct weight** for that node.

A side comment about my solution and my performance with this problem: It took me until the morning to get part two, and therefore, I've cleaned up my solution in a way that I haven't up to this point. The general strategy is the same, but I've made the code prettier and resolved a corner case that my original solution would have ignored.

A second comment about my solution: Despite the name of today's problem and the fact that this problem is begging for tree traversal, I do not use recursion in my solution. I have an iterative solution, though the general strategy could also be expressed recursively. I suspect the recursive solution is slightly more efficient.

Nonetheless, my solution is below, and I break it down afterwards.

In [4]:
def day7b():
    f = open('input\\input7.txt', 'r')
    lines = [line.strip() for line in f.readlines()]
    f.close()

    tree = dict()
    local_weights = dict()
    for line in lines:
        parts = line.split()
        parent = parts[0]
        weight = int(parts[1][1:-1])
        kids = ()
        if len(parts) >= 3:
            kids = tuple([p.rstrip(',') for p in parts[3:]])
        tree[parent] = kids
        local_weights[parent] = weight
    
    subtree_weights = local_weights.copy()
    
    verified = dict()
    for node in tree:
        verified[node] = False
    
    while not all(verified.values()):
        for parent,kids in tree.items():
            if not kids:
                verified[parent] = True
            elif all(verified[k] for k in kids) and not verified[parent]:
                kid_wgts = [subtree_weights[k] for k in kids]
                low = min(kid_wgts)
                high = max(kid_wgts)
                if low != high:
                    wrong, right = (low, high) if kid_wgts.count(low) == 1 else (high, low)
                    idx = kid_wgts.index(wrong)
                    diff = wrong - right
                    return kids[idx], local_weights[kids[idx]]-diff
                subtree_weights[parent] += sum(kid_wgts)
                verified[parent] = True
                
    return False

    
day7b()

('cumah', 1060)

After I read in the input, I set up two dictionaries to contain all of the data about my tree:
* `tree` is a mapping between a node and a tuple of that node's children
* `local_weights` is a mapping between a node and the weight of those nodes
The for loop constructs these two dictionaries from our input file.

The next step creates two more dictionaries:
* `subtree_weights` will be a mapping between a node and the weight of the subtree for which that node is the root. We will need to build these values up though. Initially, this is just a copy of the `local_weights` dictionary.
* `verified` is a mapping between a node and a boolean. A node is considered verified if its value in `subtree_weights` has been completely built up. Initially, all nodes are considered not verified.

The main logic of the program is in the while loop, where we continue until all nodes have been verified. In reality, assuming the tree is actually unbalanced, we will return from the inside of the loop before the condition is ever false. During each pass of the while loop, we iterate over all of the nodes of the tree, breaking the node entry down into itself (`parent`) and its list of children (`kids`). There are three cases to deal with:
1. The node has no kids.
2. The node has not been verified, but all of its kids have been verified.
3. The node has not been verified, and neither has at least one of its kids.

In the first case (`if not kids`), we can verify the node because the weight of its subtree is simply its own weight. Jumping to the third case, we don't want to do anything there, because we need to wait to verify all kids before attempting to verify a parent. So the magic of detecting the error node occurs in the second case (the `elif` block).

If all of the kids have had their subtree totals verified, then either this node is balanced or unbalanced. It is balanced if all of the subtree totals (contained in `kid_wgts`) are equal. If not, it is unbalanced. We verify balance by comparing the lowest and highest values in `kid_wgts`. If they are different, then the node is unbalanced and we know that the problem lies in one of its children. The `if low != high` block deals with that issue. However, let's consider what we do when the node was balanced. In that case, we can now verify this node. We update this node's subtree weight by adding the subtree weights of all of its children to it.

If the node is unbalanced, we need to figure out which child is the problem. That child is the one whose subtree weight is unique among all of the children. We can't be sure if it is the child with the lowest weight or the highest weight, so we determine the weight that is present only once in the list. Then we figure out which child has that weight. Our goal is to identify what that child's correct weight should be, so we return the difference between the child weights (`diff`) from the local weight of that child. This code also returns the name of the problem child.

This is the first problem for the year where I would say **Data Structures** is required to solve the problem. *Understanding trees and how to traverse them* is absolutely necessary, and it helps if you can recognize that the input is an *adjacency list*. Even part one requires that you understand the basic structure of a tree -- roots have no parent -- but I imagine that could have been solved without a Data Structures level of understanding of trees.

# [Day 8](http://adventofcode.com/2017/day/8): I Heard You Like Registers

For this problem, we are given a series of instructions to carry out on a set of registers. Each instruction is an if-statement: Compare a register to a value, and if the comparison evaluates to True, increase/decrease a different register by the specified amount. The difficulty is, we do not know the names of all of our registers in advance. Fortunately, we know that all registers start the problem containing zero. Thus, if we haven't seen them before, we can treat them as 0.

For **part one**, we are asked to identify the largest value in the registers at the end of the instruction set. First, let's define a function to evaluate the if-statement portion of the instruction.

In [1]:
def evaluate(registers, reg, arg, amt):
    value = registers.get(reg, 0)
    if arg == '<':
        return value < amt
    elif arg == '>':
        return value > amt
    elif arg == '<=':
        return value <= amt
    elif arg == '>=':
        return value >= amt
    elif arg == '==':
        return value == amt
    elif arg == '!=':
        return value != amt

The function above takes 4 parameters: A dictionary containing all known registers and their current values, the register in the if statement, which comparison argument was used (must be in this set: `{==, !=, <, >, <=, >=}`), and the amount to which we are comparing the register value (an integer). The first step is to retrieve the register's value from the dictionary. Using the dictionary's `get()` method ensures that we get 0 if it turns out we don't know the value of the register. The remainder of the function is a series of if statements that carry out the appropriate comparison and return `True` or `False`.

Two alternative approaches that I've seen for how to do this step, since I admit that this is a big block of code to do not all that much:
1. Using the `eval()` function, which evaluates a given line of code. So we could write `return eval(str(value)+arg+str(amt))`. While this would work out, since we still find value using the dictionary itself, I did notice on Reddit that people who tried to read from the dictionary during the eval function ran into an issue where one of the registers was named `if`, which is obviously a keyword in Python. Potentially dangerous, albeit shorter if we're competing to win.
2. Map each symbol to the special comparison functions, such as `__eq__` or `__lt__`. Perfectly reasonable. I would argue that mine is more readable, but it's not a particularly strong argument.

Regardless of strategy, we now have the ability to evaluate the conditional in each instruction. Now let's look at our main function to solve the problem:

In [2]:
def day8a():
    f = open('input\\input8.txt', 'r')
    lines = [line.strip() for line in f.readlines()]
    f.close()

    registers = {}
    for line in lines:
        data = line.split()
        reg = data[0]
        mult = 1 if data[1] == 'inc' else -1
        amount = int(data[2])
        if_reg = data[4]
        arg = data[5]
        if_amt = int(data[6])
        
        result = evaluate(registers, if_reg, arg, if_amt)
        if result:
            value = registers.get(reg, 0)
            registers[reg] = value+(mult*amount)
            
    return max(registers.values())

day8a()

5946

As mentioned above, we can keep our registers and their values in a dictionary. We start out with an empty dictionary. The first few lines of the `for` loop simply break the instructions up into parts. Let `data` be our line after we break on whitespace. Then, indices 0-2 of `data` are the register to change, whether to increase or decrease, and how much to increase/decreate, respectively. Increasing and decreasing are specified by the keyword `inc` or `dec` in the instruction. We'll specify a multiplier `mult` , which will be 1 or -1 depending on which instruction we saw. Index 3 of `data` is always `if`, which we don't actually need. Indices 4-6 are the comparison, which we'll break apart into the register, comparison operation, and amount to compare. We send those to our evaluate() function. If it returns `True`, we can carry out the operation on our registers. Again, we'll use the `get()` method with our dictionary so that we can force a value of 0 if we've never seen the register before.

Once we've gone through the list of instructions, our dictionary contains the register names as keys, and their contents as values. We return the maximum of the dictionary values as the solution to our problem.

For **part two**, we're asked to report the highest value that any register ever held during the entire instruction set. Fortunately, this requires only a small modification to my solution.

In [3]:
def day8b():
    f = open('input\\input8.txt', 'r')
    lines = [line.strip() for line in f.readlines()]
    f.close()

    max_seen = 0
    registers = {}
    for line in lines:
        data = line.split()
        reg = data[0]
        mult = 1 if data[1] == 'inc' else -1
        amount = int(data[2])
        if_reg = data[4]
        arg = data[5]
        if_amt = int(data[6])
        
        result = evaluate(registers, if_reg, arg, if_amt)
        if result:
            value = registers.get(reg, 0)
            registers[reg] = value+(mult*amount)
            if registers[reg] > max_seen:
                max_seen = registers[reg]
            
    return max_seen

day8b()

6026

A rundown of our changes:
* Prior to the loop, we add a `max_seen` value to keep track of the highest value we've seen so far. We initialize it to 0, since all registers start out at 0. It's therefore the highest we've seen. (Line 6)
* If we need to update a register, we will also check to see if its new value is larger than `max_seen`. If so, we will update `max_seen`. (Lines 21-22)
* We return `max_seen` instead of the maximum current value. (Line 24)

I would like to make this solution a little neater, so I'm going to revise a couple of pieces before summarizing. First, let's replace evaluate() with one of the other strategies. I feel a bit allergic towards the idea of using `eval()`, so I'm going to use the other approach I described above.

In [6]:
def compare(registers, reg, op, amt):
    ops = {'<':int.__lt__, '>':int.__gt__,
           '<=':int.__le__, '>=':int.__ge__,
           '!=':int.__ne__, '==':int.__eq__}
    value = registers.get(reg, 0)
    return ops[op](value,amt)

Now we have a dictionary mapping a comparison operator to the comparison functions in Python's `int` class. The return statement looks up which function to use in the dictionary and applies it to `value` and `amt`.

I'm making a couple of tweaks to my main solution function, as well:

In [7]:
def day8():
    f = open('input\\input8.txt', 'r')
    lines = [line.strip() for line in f.readlines()]
    f.close()

    max_seen = 0
    registers = {}
    for line in lines:
        data = line.split()
        reg = data[0]
        mult = 1 if data[1] == 'inc' else -1
        amount = int(data[2])
        if_reg = data[4]
        op = data[5]
        if_amt = int(data[6])
        
        if compare(registers, if_reg, op, if_amt):
            value = registers.get(reg, 0)
            registers[reg] = value+(mult*amount)
            max_seen = max(max_seen, registers[reg])
            
    return max(registers.values()), max_seen

day8()

(5946, 6026)

These changes were:
* Renaming the `arg` variable to `op`. (It's really an operator, not an argument.)
* Put the call to the new `compare()` function in the `if` condition, instead of on a separate line. (Line 17)
* Replaced the `if` block that decided whether to update `max_seen` with a call to `max()` which makes the same comparison as before. (Line 20)
* Return the values needed for Part 1 and Part 2. (Line 22)

In terms of difficulty, I haven't used any data structures tricks in this solution (any version of it). My final solution definitely employs some deep syntactic knowledge of Python though, beyond what I would expect an introductory student to know. But I think the earlier solutions are within the scope of an **Introduction to Programming** student. The hardest parts about this for an intro student are, in my mind:
* The sheer amount of *string manipulation* required.
* Deciding how to handle the `inc` or `dec` instruction.
* Evaluating the conditional part of the instruction. I think new programmers would attempt to use the comparison operator as a literal operator in the code, not recognizing immediately that it's a string that Python won't interpret as an operation.
* Knowing how to make sure that new registers are treated as containing zero. This requires an understanding of how *dictionary retrieval* works in Python.

I think this is within the range of a student at the end of Introduction to Programming, though this would be a project for them. More approachable than Day 7, but requiring substantial effort for new programmers.

# [Day 9](http://adventofcode.com/2017/day/9): Stream Processing

For this problem, we are given a single line of input. Lines are made up of groups, encased in `{ }`, and garbage, encased in `< >`. Groups can be nested inside each other, but garbage is not nested. Curly braces inside the garbage symbols can be ignored, as they do not indicate the start of groups. Finally, the `!` character, when it is part of garbage, tells us to ignore the next character.

For **part one**, we are instructed asked to calculate the total *score* of the groups in our input, where an individual group's score is based on how nested it is. The outermost group has a score of 1. All other groups have a score 1 greater than the groups they are nested in. Solution code below:

In [1]:
def day9a():
    f = open('input\\input9.txt', 'r')
    data = [line.strip() for line in f.readlines()][0]
    f.close()

    depth = 0
    index = 0
    total = 0
    while index < len(data):
        ch = data[index]
        if ch == '{':
            depth += 1
            total += depth
            index += 1
        elif ch == '}':
            depth -= 1
            index += 1
        elif ch == '<':
            index += 1
            while data[index] != '>':
                if data[index] == '!':
                    index += 2
                else:
                    index += 1
            index += 1
        else:
            index += 1
        
    return total

day9a()

21037

We will keep track of three variables inside our loop:
- `depth` tracks how nested we are inside groups at this point
- `index` tracks where we are in our input. We could set up a `for` loop to iterate over our input by character or index, but we will get greater control over this index by using a `while` loop.
- `total` represents the total score, the goal for our problem.

Once we get inside the loop, there are four cases we need to consider for each character we look at:
- **Case 1:** We encounter a `{`, representing the beginning of a group. We increase our depth by 1 and add depth to our total. We can advance the index 1 spot in our input.
- **Case 2:** We encounter a `}`, representing the end of a group. We decrease our depth by 1. No change to our total, because we track it only when we see open braces. We can advance the index 1 space.
- **Case 3:** We encounter a `<`, representing the beginning of garbage. We know we stay in garbage until we hit a `>`, unless that `>` came immediately after a `!`. So we will start another loop that advance the index until we hit that `>`, skipping an extra space if we encounter a `!`. Once we find the `>`, we move on to the next character.
- **Case 4:** Literally anything else. Just move to the next character.

One note: In my initial solution, I thought we had to deal with `!` characters outside of garbage as well. I had a case for skipping an extra character when we hit one of those. Fortunately, my input never caused this to be a problem.

For **part two**, we are asked to report how many characters are inside the garbage. The opening and closing braces `< >` do not count, nor do `!` or the character after them that gets canceled out. We can solve this problem with just a few minor tweaks.
* Add a `garbage` counter. (Line 9)
* Increment the `garbage` counter only in the inner loop of *Case 3* above, when we are not looking at a `!` character. (Line 26)

Solution below, returning results from both parts:

In [2]:
def day9b():
    f = open('input\\input9.txt', 'r')
    data = [line.strip() for line in f.readlines()][0]
    f.close()

    depth = 0
    index = 0
    total = 0
    garbage = 0
    while index < len(data):
        ch = data[index]
        if ch == '{':
            depth += 1
            total += depth
            index += 1
        elif ch == '}':
            depth -= 1
            index += 1
        elif ch == '<':
            index += 1
            while data[index] != '>':
                if data[index] == '!':
                    index += 2
                else:
                    index += 1
                    garbage += 1
            index += 1
        else:
            index += 1
        
    return total, garbage

day9b()

(21037, 9495)

As I initally thought about this problem, I immediately jumped to using a stack to solve it, as we would for many parentheses-matching problems. However, that is overkill for two reasons: 
1. We are guaranteed that our input stream is well-formed, including every group and garbage pile within it. We're not trying to verify the format, just answer questions about it.
2. We only care about the depth of groups. We don't need to check how nested garbage is inside those groups.
With those conditions, we can track depth without a stack using an integer, knowing that depth increases or decreases at each brace.

A **Data Structures** student could absolutely solve this problem, using a stack or not. The hardest part of this problem for an **Introduction to Programming** student would be setting up the loops. I used a `while` loop for the outer loop in order to get finer control over the `index` variable. (This is a side-effect of how Python sets up `for` loops. In Java, I would have used a `for` loop, because I would have had the same control.) For the inner loop, defining the end condition and making sure the `!` escape character is handled properly are the hardest parts. I think the knowledge is there for an intro student, but *knowledge of stacks* could make this problem easier.

# [Day 10](http://adventofcode.com/2017/day/10): Knot Hash

For this problem, our input represents a set of instructions for a hash function. While I've tried to summarize past problems in my own words, I don't think I can do so easily for this one. So instead, I will provide the main instructions directly from the problem:

>To achieve this, begin with a list of numbers from 0 to 255, a **current position** which begins at 0 (the first element in the list), a **skip size** (which starts at 0), and a sequence of **lengths** (your puzzle input). Then, for each length:

>- Reverse the order of that length of elements in the list, starting with the element at the current position.
>- Move the current position forward by that length plus the skip size.
>- Increase the skip size by one.

The list is circular, so we need to be able to reverse around the end of the list when necessary. For **part one**, we are asked to provide the product of the first two elements of the resulting list.

Of the 3 bullets above, reversal is clearly the most difficult part, so I will begin with my reverse function.

In [2]:
def reverse(mylist, start, amt):
    if start+amt > len(mylist):
        diff = (start+amt)-len(mylist)
        reversable = mylist[start:]+mylist[:diff]
        reversed = reversable[::-1]
        return reversed[-diff:]+mylist[diff:start]+reversed[:-diff]
    else:
        end = start+amt
        return mylist[:start]+mylist[end-1:start:-1]+[mylist[start]]+mylist[end:]

Given a list, a starting point, and an amount indicating how much of the list to reverse, return the reversed list. I break the task down into two cases: Either we need to wrap around to the front of the list, or we do not. In the first case, we determine how far we need to go around the front of the list. We build `reversable`, which is built from the portions of the list we need to reverse. The next line does that reversal. But now we need to build the new list, including the unreversed section, and making sure to reattach whatever portion we grabbed from the front. The last `diff` items of `reversed` are the part we stole from the front. From there, we can grab the untouched section. Finally, we concatenate the rest of `reversed`.

In the second case, we know the section reverse is in the middle. So we leave the untouched ends bookended, and do the reversal right in the middle.

The remainder of the work is in my main function:

In [3]:
def day10a():
    f = open('input\\input10.txt', 'r')
    data = f.readline()
    f.close()
    
    lens = [int(x) for x in data.split(',')]
    
    mylist = list(range(256))
    current = 0
    skip = 0
    
    for i in lens:
        mylist = reverse(mylist, current, i)
        current += (i + skip)
        current = current%len(mylist)
        skip += 1
        
    return mylist[0]*mylist[1]

day10a()

9656

In the main for loop, we follow the bullet points spelled out earlier. Reverse the appropriate section of the list. Update current by adding the length and skip size, making sure to keep current within the length of the list. Finally, we increment the skip size.

In **part two**, we change a lot of the details of the problem. The algorithm that we just worked out is still the same, but we use it in a different way. First, our input is no longer treated as a sequence of numbers, but a sequence of characters, commas included. Each character needs to be converted into its ASCII value, and we attach a few additional values to our input. Second, we run our algorithm above 64 times, preserving the current index and skip size after each round. Once that is complete, we take our resulting *sparse hash* and reduce it by XOR-ing each set of 16 values. Thus, our 256 elements become 16 elements. Finally, we return the hex representation of those 16 elements.

First, a function to XOR 16 elements of a list:

In [4]:
import functools

def xor(mylist,start,end):
    return functools.reduce(lambda x, y: x^y, mylist[start:end])

Now, our overall solution, built from our first part, but with some real changes too.

In [7]:
def day10b():
    f = open('input\\input10.txt', 'r')
    data = f.readline()
    f.close()
    
    lens = [ord(x) for x in data]
    lens += [17,31,73,47,23]
    
    mylist = list(range(256))
    current = 0
    skip = 0
    
    for rd in range(64):
        for i in lens:
            mylist = reverse(mylist, current, i)
            current += (i + skip)
            current = current%len(mylist)
            skip += 1

    dense = [xor(mylist,a,a+16) for a in range(0,len(mylist),16)]
    result = [hex(j)[2:] for j in dense]        
    return ''.join(result)

print(day10b())

20b7b54c92bf73cf3e5631458a715149


A summary of changes from part one to part two:
* In lines 6-7, we modify our input to be a sequence of ASCII values, rather than the integers they were in part one. We also tack on the additional values specified by the problem.
* Line 13 sets up our loop to run the process from part one 64 times.
* Line 20 does the XOR process in chunks of 16, while Line 21 converts each remaining element to hex. We use the `[2:]` to chop of the `0x` that comes in front of any hex string. Line 22 turns the list of hex strings into a single string.

I'm having trouble placing the difficulty on this problem. Personally, I struggled on the reverse() function (debugging it kept me from the leaderboard), so I imagine it would be troublesome for any intro to programming students, though I didn't use anything I don't teach in that class. I also made use of several language features that I normally don't cover in Intro or Data Structures, notably the XOR operator `^` and `functools.reduce()`. That said, `reduce()` was a modification that I made after submission. Originally, I used a `for` loop in my XOR function. This is hard to place, but I would maybe say at least **Data Structures** for some programming experience.